# Imports

In [10]:
import os, sys, time, glob, random, argparse
import wandb
import numpy as np
from copy import deepcopy
import torch
import torch.nn as nn
import time
import tqdm
import scipy.stats as stats
import matplotlib.pyplot as plt
import pickle
import pandas as pd

# XAutoDL 
from xautodl.config_utils import load_config, dict2config, configure2str
from xautodl.datasets import get_datasets, get_nas_search_loaders
from xautodl.procedures import (
    prepare_seed,
    prepare_logger,
    save_checkpoint,
    copy_checkpoint,
    get_optim_scheduler,
)
from xautodl.utils import get_model_infos, obtain_accuracy
from xautodl.log_utils import AverageMeter, time_string, convert_secs2time
from xautodl.models import get_search_spaces

# API
from nats_bench import create

# custom modules
from custom.tss_model import TinyNetwork
from xautodl.models.cell_searchs.genotypes import Structure
from ZeroShotProxy import *
from tss_utils import compute_vkdnw, plot_stats, get_stats, get_metrics, analyze_results, generate_accs, get_results_from_api, get_scores

# All

In [11]:
# scp 'tyblondr@login.rci.cvut.cz:~/PycharmProjects/VKDNW/NB201/*_log.csv' data_0811/

target = 'val_accs'
dataset = 'ImageNet16-120'
compute_graf = True
zero_cost_score_list = ['vkdnw', 'vkdnw_dim', 'vkdnw_chisquare', 'az_nas', 'jacov','gradsign', 'zico', 'zen', 'grad_norm', 'naswot', 'synflow', 'snip', 'grasp', 'te_nas', 'flops']
plt.rcParams['text.usetex'] = False
compute_graf_str = 'filtered' if compute_graf else 'unfiltered'

In [12]:
run_dict = {
    'cifar10': {
        # same images
        #'nazderaze/VKDNW/n6n44keg': 1,
        #'nazderaze/VKDNW/6c1pv095': 2,
        #'nazderaze/VKDNW/45us11be': 3,
        #'nazderaze/VKDNW/rwp2qvkw': 4,
        #'nazderaze/VKDNW/6bsssh4z': 5,
        
        # different images
        #'nazderaze/VKDNW/7a8jm975' : -1 # testing (successfull)
        #'nazderaze/VKDNW/cixw6r9b': 1,
        #'nazderaze/VKDNW/1nikalf3': 2,
        #'nazderaze/VKDNW/r5l3qg12': 3,
        #'nazderaze/VKDNW/fq581h0d': 4,
        #'nazderaze/VKDNW/0yfzaxra': 5,
        
        # entropy
        #'nazderaze/VKDNW/87vy5rby': 1,
        #'nazderaze/VKDNW/8sd2hdws': 5,
        #'nazderaze/VKDNW/bh09isa0': 3,
        #'nazderaze/VKDNW/ei31s875': 4,
        #'nazderaze/VKDNW/koxirqt3': 2,
        
        # entropy all
        #'nazderaze/VKDNW/6lcy0jqp': 1,
        #'nazderaze/VKDNW/orl2me1l': 2,
        #'nazderaze/VKDNW/y5n4lnsn': 3,
        #'...': 4,
        #'nazderaze/VKDNW/7gkdl3gl': 5,
        
        # final
        'nazderaze/VKDNW/k7llaf2s': 1,
        'nazderaze/VKDNW/knu2gv65': 2,
        'nazderaze/VKDNW/fnfbpspk': 3,
        'nazderaze/VKDNW/w4b39d0d': 4,
        'nazderaze/VKDNW/vbr5n8iy': 5
        
    },
    'cifar100': {
        # same images
        #'nazderaze/VKDNW/qmq5vp3k': 1,
        #'nazderaze/VKDNW/31lrq6p7': 2,
        #'nazderaze/VKDNW/3qazc6po': 3,
        #'nazderaze/VKDNW/424twoyv': 4,
        #'nazderaze/VKDNW/783h4opf': 5,
        
        # different images
        #'nazderaze/VKDNW/n2m8i53l': 1,
        #'nazderaze/VKDNW/48xbnfdg': 2,
        #'nazderaze/VKDNW/f0czkx5u': 3,
        #'nazderaze/VKDNW/ldv3b1bh': 4,
        #'nazderaze/VKDNW/qwae4nqx': 5,
        
        # entropy
        #'nazderaze/VKDNW/lfnin2ui': 1,
        #'nazderaze/VKDNW/1cs8orlm': 4,
        #'nazderaze/VKDNW/4tep5lv0': 5,
        #'nazderaze/VKDNW/lqka0nwl': 2,
        #'nazderaze/VKDNW/mqrfyv7t': 3,
        
        # entropy all
        #'...': 1,
        #'...': 2,
        #'...': 3,
        #'...': 4,
        #'...': 5,
        
        # final
        'nazderaze/VKDNW/v2b816ul': 1,
        'nazderaze/VKDNW/velwxtxd': 2,
        'nazderaze/VKDNW/f3ljf5jf': 3,
        'nazderaze/VKDNW/e3w3dkv7': 4,
        'nazderaze/VKDNW/9ibvj04q': 5,
    },
    'ImageNet16-120': {
        # same images
        #'nazderaze/VKDNW/ftg0tdsa': 1,
        #'nazderaze/VKDNW/vqf1ey6x': 2,
        #'nazderaze/VKDNW/v0a0m67q': 3,
        #'nazderaze/VKDNW/uiv37u18': 4,
        #'nazderaze/VKDNW/c1338vfg': 5,
        
        # different images
        #'nazderaze/VKDNW/55f1omxn': 1,
        #'nazderaze/VKDNW/amdcxrz7': 2,
        #'nazderaze/VKDNW/sl0rjhwh': 3,
        #'nazderaze/VKDNW/z2ph6iav': 4,
        #'nazderaze/VKDNW/ol9rwkeo': 5,
        
        # entropy
        # 'nazderaze/VKDNW/hwtw58ot': -1 # test
        #'nazderaze/VKDNW/u214u8p1': 4,
        #'nazderaze/VKDNW/bycu6ed3': 5,
        #'nazderaze/VKDNW/2ztcsos6': 3,
        #'nazderaze/VKDNW/kptcxbi2': 2,
        #'nazderaze/VKDNW/f9n7j83e': 1,
        
        #'nazderaze/VKDNW/hrrxcexh': 1,
        #'nazderaze/VKDNW/jwhy017w': 2,
        #'nazderaze/VKDNW/82rycftz': 3,
        #'nazderaze/VKDNW/bw6b93p9': 4,
        #'nazderaze/VKDNW/ca423k7t': 5,
        
        # final
        'nazderaze/VKDNW/ss2kwvpp': 1,
        'nazderaze/VKDNW/es9t2696': 2,
        'nazderaze/VKDNW/m1dusbs9': 3,
        'nazderaze/VKDNW/kgmcpu2y': 4,
        'nazderaze/VKDNW/cpbmsvuw': 5,
        
    }
}

In [13]:
api_nats = create('/mnt/personal/tyblondr/NATS-tss-v1_0-3ffb9-simple/', 'tss', fast_mode=True, verbose=False)

if os.path.exists(f"./tss_features_{dataset}.pickle"):
    archs = pd.read_pickle(f"./tss_features_{dataset}.pickle")
else:
    archs = generate_accs(api_nats, dataset=dataset)
    print(f'No. of generated archs: {archs.shape[0]}')
    archs.to_pickle(f"./tss_features_{dataset}.pickle")

In [14]:
api_wandb = wandb.Api()

log = None
results = None
for run_id, seed in run_dict[dataset].items():
    
    run = pd.DataFrame(api_wandb.run(run_id).scan_history())
    run.rename({'arch': 'net_str'}, axis=1, inplace=True)
    
    run = pd.merge(archs, run, on='net_str', how='inner')
    if compute_graf:
        run = run.loc[run['net'].notnull(), :]  # keep only nets with features
        
    print(f'No. of archs for seed {seed} after filtering: {run.shape[0]}.')
    
    for col in run.columns:
        if col not in ['net_str', 'net']:
            run[col] = run[col].astype(float)
    
    if 'jacov' in run.columns:        
        run['jacov'] = run['jacov'].fillna(run['jacov'].min()).astype(float)

    df_scores = get_scores(run.copy(), compute_graf=compute_graf, zero_cost_score_list=zero_cost_score_list)
    df_scores['dataset'] = dataset
    df_scores['seed'] = seed
    if results is None:
        results = df_scores
    else:
        results = pd.concat([results, df_scores], ignore_index=True)
print(f'Total number of records: {results.shape[0]}')

No. of archs for seed 1 after filtering: 9445.
Running vkdnw
Running vkdnw_dim
Running vkdnw_chisquare
Running az_nas
Running jacov
Running gradsign
Running zico
Running zen
Running grad_norm
Running naswot
Running synflow
Running snip
Running grasp
Running te_nas
Running flops
No. of archs for seed 2 after filtering: 9445.
Running vkdnw
Running vkdnw_dim
Running vkdnw_chisquare
Running az_nas
Running jacov
Running gradsign
Running zico
Running zen
Running grad_norm
Running naswot
Running synflow
Running snip
Running grasp
Running te_nas
Running flops
No. of archs for seed 3 after filtering: 9445.
Running vkdnw
Running vkdnw_dim
Running vkdnw_chisquare
Running az_nas
Running jacov
Running gradsign
Running zico
Running zen
Running grad_norm
Running naswot
Running synflow
Running snip
Running grasp
Running te_nas
Running flops
No. of archs for seed 4 after filtering: 9445.
Running vkdnw
Running vkdnw_dim
Running vkdnw_chisquare
Running az_nas
Running jacov
Running gradsign
Running zico
R

In [15]:
log = None
for seed in results['seed'].unique():
    
    results_temp = results.loc[results['seed'] == seed, :].copy()
    for zero_cost_rank in [p for p in results_temp.columns if '_rank' in p]:
        results_temp[[zero_cost_rank]] = results_temp[[zero_cost_rank]].apply(lambda x: x.replace(-np.inf, x[x != -np.inf].min()))
        results_temp[[zero_cost_rank]] = results_temp[[zero_cost_rank]].apply(lambda x: x.replace(-np.nan, x[x != -np.inf].min()))
        results_temp[[zero_cost_rank]] = results_temp[[zero_cost_rank]].apply(lambda x: x.replace(np.inf, x[x != np.inf].max()))
        log_temp = pd.DataFrame(get_metrics(results_temp, pred_name=zero_cost_rank, show_plot=False, seed=seed))
        
        if log is None:
            log = log_temp.copy()
        else:
            log = pd.concat([log, log_temp.copy()], ignore_index=True)
            
        if seed == min(results['seed'].unique()):
            plot_stats(get_stats(results_temp, 'vkdnw_dim', target, zero_cost_rank), 'vkdnw_dim', target, zero_cost_rank, f'{dataset}_{str(compute_graf)}_{zero_cost_rank}')

log = log.groupby('pred_name', as_index=False).agg(['mean', 'std']).reset_index()
log['dataset'] = dataset
log['no_seeds'] = len(results['seed'].unique())
log['archs_filtered'] = compute_graf_str
log

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript back

pred_name   kendall            spearman             pearson  \
                              mean       std      mean       std      mean   
0            az_nas_rank  0.672816  0.001249  0.858689  0.001364  0.873234   
1             flops_rank  0.545322  0.000000  0.717813  0.000000  0.593022   
2         grad_norm_rank  0.310441  0.000504  0.417613  0.000802  0.036808   
3          gradsign_rank -0.454410  0.002430 -0.626827  0.003195 -0.679063   
4             grasp_rank  0.359110  0.003922  0.501660  0.006311 -0.000345   
5             jacov_rank  0.602018  0.001780  0.779219  0.001888  0.399077   
6            naswot_rank  0.605221  0.000404  0.794006  0.000386  0.815677   
7              snip_rank  0.389260  0.000437  0.521150  0.000602  0.101274   
8           synflow_rank  0.531145  0.000146  0.718543  0.000091  0.081943   
9            te_nas_rank  0.522621  0.001744  0.709051  0.001982  0.669929   
10  vkdnw_chisquare_rank  0.149948  0.003749  0.228106  0.004529  0.198410   
11       vkdnw_comb_rank  0.742428  0.000472  0.905600  0.000335  0.916114   
12        vkdnw_dim_rank  0.622966  0.000000  0.763595  0.000000  0.747485   
13        vkdnw_exp_rank  0.676885  0.001133  0.863611  0.001130  0.878307   
14       vkdnw_prog_rank  0.720608  0.000346  0.895149  0.000262  0.899860   
15            vkdnw_rank  0.615831  0.000842  0.808589  0.000777  0.805751   
16      vkdnw_train_rank  0.652615  0.001421  0.832237  0.001307  0.851373   
17              zen_rank  0.091004  0.000221  0.108974  0.000380  0.079410   
18             zico_rank  0.586679  0.000583  0.778793  0.000544  0.756605   

                  ndcg              acc_top            acc_top_true       \
         std      mean       std       mean        std         mean  std   
0   0.001141  0.982959  0.000359  44.792222   1.361551    47.311111  0.0   
1   0.000000  0.965469  0.000000  41.444444   0.000000    47.311111  0.0   
2   0.008416  0.921969  0.000261  18.450000  10.127342    47.311111  0.0   
3   0.001059  0.755702  0.000599  12.208889   1.132227    47.311111  0.0   
4   0.043452  0.920355  0.001044  12.051111  10.259637    47.311111  0.0   
5   0.019510  0.971225  0.000376  38.788889   1.119282    47.311111  0.0   
6   0.000147  0.972982  0.000043  37.108889   3.647275    47.311111  0.0   
7   0.009506  0.923113  0.000402   0.833333   0.000000    47.311111  0.0   
8   0.000738  0.963887  0.000088  40.782222   4.723337    47.311111  0.0   
9   0.002771  0.957454  0.000906  39.746667   0.789208    47.311111  0.0   
10  0.006884  0.890923  0.000764  40.002222   3.242638    47.311111  0.0   
11  0.000136  0.986205  0.000089  44.466667   1.270414    47.311111  0.0   
12  0.000000  0.971704  0.000000  35.477778   0.000000    47.311111  0.0   
13  0.000433  0.982739  0.000442  45.067778   1.261224    47.311111  0.0   
14  0.000299  0.985934  0.000091  44.584444   1.213774    47.311111  0.0   
15  0.001539  0.974066  0.000238  44.196667   1.951739    47.311111  0.0   
16  0.001372  0.977148  0.000415  41.793333   0.871093    47.311111  0.0   
17  0.000467  0.866953  0.000128  40.766667   0.000000    47.311111  0.0   
18  0.000099  0.972960  0.000078  41.444444   0.000000    47.311111  0.0   

           dataset no_seeds archs_filtered  
                                            
0   ImageNet16-120        5       filtered  
1   ImageNet16-120        5       filtered  
2   ImageNet16-120        5       filtered  
3   ImageNet16-120        5       filtered  
4   ImageNet16-120        5       filtered  
5   ImageNet16-120        5       filtered  
6   ImageNet16-120        5       filtered  
7   ImageNet16-120        5       filtered  
8   ImageNet16-120        5       filtered  
9   ImageNet16-120        5       filtered  
10  ImageNet16-120        5       filtered  
11  ImageNet16-120        5       filtered  
12  ImageNet16-120        5       filtered  
13  ImageNet16-120        5       filtered  
14  ImageNet16-120        5       filtered  
15  ImageNet16

In [16]:
"""
for zero_cost_score in ['vkdnw']:
    for i in range(5):
        results_temp = results.loc[results['seed'] == results['seed'].unique().min(), :].copy().sample(n=3000)
        analyze_results(api_nats, results_temp, zero_cost_score, target)
        
df_top = None
for seed in range(1, 6):
    results_temp = results.loc[results['seed'] == seed, :].sample(n=3000, random_state=seed).copy()
    for pred in [p for p in results_temp.columns if '_rank' in p]:
        top_acc = results_temp.loc[results_temp[pred].idxmax(), target]
        if df_top is None:
            df_top = pd.DataFrame({'acc': top_acc, 'acc_max': results_temp[target].max(), 'pred': pred, 'seed': seed}, index=[0])
        else:
            df_top = pd.concat([df_top, pd.DataFrame({'acc': top_acc, 'acc_max': results_temp[target].max(), 'pred': pred, 'seed': seed}, index=[0])], ignore_index=True)
df_top.groupby('pred', as_index=False)[['acc', 'acc_max']].agg(['max', 'mean', 'std']).reset_index()
"""

"\nfor zero_cost_score in ['vkdnw']:\n    for i in range(5):\n        results_temp = results.loc[results['seed'] == results['seed'].unique().min(), :].copy().sample(n=3000)\n        analyze_results(api_nats, results_temp, zero_cost_score, target)\n        \ndf_top = None\nfor seed in range(1, 6):\n    results_temp = results.loc[results['seed'] == seed, :].sample(n=3000, random_state=seed).copy()\n    for pred in [p for p in results_temp.columns if '_rank' in p]:\n        top_acc = results_temp.loc[results_temp[pred].idxmax(), target]\n        if df_top is None:\n            df_top = pd.DataFrame({'acc': top_acc, 'acc_max': results_temp[target].max(), 'pred': pred, 'seed': seed}, index=[0])\n        else:\n            df_top = pd.concat([df_top, pd.DataFrame({'acc': top_acc, 'acc_max': results_temp[target].max(), 'pred': pred, 'seed': seed}, index=[0])], ignore_index=True)\ndf_top.groupby('pred', as_index=False)[['acc', 'acc_max']].agg(['max', 'mean', 'std']).reset_index()\n"

In [17]:
log_train = None
if not compute_graf:
    print('No graf prediction.')
else:
    print('Graf prediction.')
    
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.pipeline import Pipeline
    from sklearn.preprocessing import StandardScaler
    from sklearn.model_selection import train_test_split
    
    pred_lists = {
        'model_vkdnw': [p for p in results.columns if '_lambda_' in p] + ['vkdnw_dim', 'flops'], #['vkdnw_chisquare', 'vkdnw_dim', 'flops'],
        'model_vkdnw+zs': [p for p in results.columns if '_lambda_' in p] + ['vkdnw_dim', 'flops'] + ['expressivity', 'progressivity', 'trainability', 'jacov','gradsign', 'zico', 'zen', 'grad_norm', 'naswot', 'synflow', 'snip', 'grasp', 'ntk', 'linear_region'],
        'model_vkdnw+zs+graf': [p for p in results.columns if '_lambda_' in p] + ['vkdnw_dim', 'flops'] + ['expressivity', 'progressivity', 'trainability', 'jacov','gradsign', 'zico', 'zen', 'grad_norm', 'naswot', 'synflow', 'snip', 'grasp', 'ntk', 'linear_region'] + [p for p in results.columns if 'op_' in p] + [p for p in results.columns if 'node_' in p],
        'model_graf': [p for p in results.columns if 'op_' in p] + [p for p in results.columns if 'node_' in p]
    }
    
    for train_size in [1024]:
        for seed in results['seed'].unique():
            
            results_temp = results.loc[results['seed'] == seed, :].copy()
            results_temp = results_temp.apply(lambda x: x.replace(-np.inf, x[x != -np.inf].min()))
            results_temp = results_temp.apply(lambda x: x.replace(np.inf, x[x != np.inf].max()))
            for model_name, pred_list in pred_lists.items():
                
                train_df, test_df = train_test_split(results_temp, test_size=1 - (train_size / results_temp.shape[0]), random_state=seed)
                model = Pipeline([
                    ('scaler', StandardScaler()),           # Step 1: Standardize features
                    ('regressor', RandomForestRegressor(n_estimators=300))  # Step 2: Train RandomForestRegressor
                ])
                model.fit(train_df[pred_list], train_df[target])
                test_df['pred_' + model_name] = model.predict(test_df[pred_list])
                log_train_temp = pd.DataFrame(get_metrics(test_df, 'pred_' + model_name, show_plot=False, seed=seed))        
                log_train_temp['train_size'] = train_size
            
                if log_train is None:
                    log_train = log_train_temp.copy()
                else:
                    log_train = pd.concat([log_train, log_train_temp.copy()], ignore_index=True)
                    
                if seed == min(results['seed'].unique()):
                    plot_stats(get_stats(test_df, 'vkdnw_dim', target, 'pred_' + model_name), 'vkdnw_dim', target, model_name, f'{dataset}_{str(compute_graf)}_{model_name}_{train_size}')

    log_train = log_train.groupby(['pred_name', 'train_size'], as_index=False).agg(['mean', 'std']).reset_index()
    log_train['dataset'] = dataset
    log_train['no_seeds'] = len(results['seed'].unique())
    log_train['archs_filtered'] = compute_graf_str
    log_train

Graf prediction.


The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


In [18]:
if compute_graf:
    log_save = pd.concat([log, log_train], ignore_index=True)
else:
    log_save = log
    
log_save.columns = ['_'.join(col) for col in log_save.columns]
log_save.to_csv(f'{dataset}_{compute_graf_str}_log.csv', index=False)
log_save

,pred_name_,kendall_mean,kendall_std,spearman_mean,spearman_std,pearson_mean,pearson_std,ndcg_mean,ndcg_std,acc_top_mean,acc_top_std,acc_top_true_mean,acc_top_true_std,dataset_,no_seeds_,archs_filtered_,train_size_
0,az_nas_rank,0.672816,0.001249,0.858689,0.001364,0.873234,0.001141,0.982959,0.000359,44.792222,1.361551,47.311111,0.0,ImageNet16-120,5,filtered,NaN
1,flops_rank,0.545322,0.000000,0.717813,0.000000,0.593022,0.000000,0.965469,0.000000,41.444444,0.000000,47.311111,0.0,ImageNet16-120,5,filtered,NaN
2,grad_norm_rank,0.310441,0.000504,0.417613,0.000802,0.036808,0.008416,0.921969,0.000261,18.450000,10.127342,47.311111,0.0,ImageNet16-120,5,filtered,NaN
3,gradsign_rank,-0.454410,0.002430,-0.626827,0.003195,-0.679063,0.001059,0.755702,0.000599,12.208889,1.132227,47.311111,0.0,ImageNet16-120,5,filtered,NaN
4,grasp_rank,0.359110,0.003922,0.501660,0.006311,-0.000345,0.043452,0.920355,0.001044,12.051111,10.259637,47.311111,0.0,ImageNet16-120,5,filtered,NaN
5,jacov_rank,0.602018,0.001780,0.779219,0.001888,0.399077,0.019510,0.971225,0.000376,38.788889,1.119282,47.311111,0.0,ImageNet16-120,5,filtered,NaN
6,naswot_rank,0.605221,0.000404,0.794006,0.000386,0.815677,0.000147,0.972982,0.000043,37.108889,3.647275,47.311111,0.0,ImageNet16-120,5,filtered,NaN
7,snip_rank,0.389260,0.000437,0.521150,0.000602,0.101274,0.009506,0.923113,0.000402,0.833333,0.000000,47.311111,0.0,ImageNet16-120,5,filtered,NaN
8,synflow_rank,0.531145,0.000146,0.718543,0.000091,0.081943,0.000738,0.963887,0.000088,40.782222,4.723337,47.311111,0.0,ImageNet16-120,5,filtered,NaN
9,te_nas_rank,0.522621,0.001744,0.709051,0.001982,0.669929,0.002771,0.957454,0.000906,39.746667,0.789208,47.311111,0.0,ImageNet16-120,5,filtered,NaN
